In [1]:
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
# import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import KFold

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
# from catboost import CatBoostRegressor

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
from torch.utils.data import DataLoader

# for Torch hyper parameter tuning
from functools import partial
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
# from ray import tune
# from ray.tune import CLIReporter
# from ray.tune.schedulers import ASHAScheduler

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

pd.set_option('display.max_columns', None)

In [2]:
# GPU device
# GPU 확인
print(torch.cuda.is_available())
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.get_device_name())
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU
1


In [3]:
import time

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

In [4]:
# Read the MPG dataset.
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", na_values=["NA", "?"]
)

cars = df["name"]

# Handle missing value
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

# Pandas to Numpy
x = df[
    [
        "cylinders",
        "displacement",
        "horsepower",
        "weight",
        "acceleration",
        "year",
        "origin",
    ]
].values
y = df["mpg"].values  # regression

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42
)

# Numpy to Torch Tensor
x_train = torch.tensor(x_train, device=device, dtype=torch.float32)
y_train = torch.tensor(y_train, device=device, dtype=torch.float32)

x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
y_test = torch.tensor(y_test, device=device, dtype=torch.float32)

In [5]:
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [6]:
# Create datasets
BATCH_SIZE = 64

dataset_train = TensorDataset(x_train, y_train)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

class Model(nn.Module):
    def __init__(self, input_dim):

        super(Model, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(input_dim, 50), 
            nn.ReLU(), 
            nn.Linear(50, 25), 
            nn.ReLU(), 
            nn.Linear(25, 1)
        )
        
    def forward(self, x):
        out = self.seq(x)
        return out
    
# Create model using nn.Sequential
model = Model(x_train.shape[1]).to(device)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

# adjust learning rate every 50 epochs
scheduler = StepLR(optimizer, step_size=50, gamma=0.90)  
loss_fn = nn.MSELoss()

# Early Stopping variables
best_loss = float('inf')
early_stopping_counter = 0
# Training loop
EPOCHS = 500
epoch = 0
done = False
es = EarlyStopping(patience=20)

# variables for plot
train_losses = []
val_losses = []

while epoch < EPOCHS and not done:
    epoch += 1
    steps = list(enumerate(dataloader_train))
    pbar = tqdm.tqdm(steps)
    model.train()
    
      for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch).flatten()  #
        loss = loss_fn(y_batch_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss)
        
        
        model.eval()
        with torch.no_grad():
            y_val_pred = model(x_batch)
        
        loss, current = loss.item(), (i + 1) * len(x_batch)
        if i == len(steps) - 1:
            model.eval()
            pred = model(x_test).flatten()
            vloss = loss_fn(pred, y_test)
            if es(model, vloss):
                done = True
            pbar.set_description(
                f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, EStop:[{es.status}]"
            )
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")
    
    
    for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch).flatten()  #
        loss = loss_fn(y_batch_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         loss, current = loss.item(), (i + 1) * len(x_batch)
#         if i == len(steps) - 1:
#             model.eval()
#             pred = model(x_test).flatten()
#             vloss = loss_fn(pred, y_test)
#             if es(model, vloss):
#                 done = True
#             pbar.set_description(
#                 f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, EStop:[{es.status}]"
#             )
#         else:
#             pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")

Epoch: 1, tloss: 903.2286987304688, vloss: 2272.873047, EStop:[]: 100%|█| 5/5 [00:00<
Epoch: 2, tloss: 1400.2303466796875, vloss: 466.656952, EStop:[Improvement found, cou
Epoch: 3, tloss: 970.4819946289062, vloss: 1087.562622, EStop:[No improvement in the 
Epoch: 4, tloss: 315.5686950683594, vloss: 200.340042, EStop:[Improvement found, coun
Epoch: 5, tloss: 533.8178100585938, vloss: 502.337952, EStop:[No improvement in the l
Epoch: 6, tloss: 220.15274047851562, vloss: 144.262131, EStop:[Improvement found, cou
Epoch: 7, tloss: 314.6019287109375, vloss: 281.069458, EStop:[No improvement in the l
Epoch: 8, tloss: 129.4928436279297, vloss: 136.936859, EStop:[Improvement found, coun
Epoch: 9, tloss: 245.60523986816406, vloss: 186.171310, EStop:[No improvement in the 
Epoch: 10, tloss: 150.37982177734375, vloss: 145.852585, EStop:[No improvement in the
Epoch: 11, tloss: 158.0860595703125, vloss: 133.723633, EStop:[Improvement found, cou
Epoch: 12, tloss: 160.00144958496094, vloss: 139.20504

Epoch: 96, tloss: 36.86466979980469, vloss: 35.363331, EStop:[No improvement in the l
Epoch: 97, tloss: 32.5416374206543, vloss: 33.412949, EStop:[Improvement found, count
Epoch: 98, tloss: 37.875789642333984, vloss: 35.405533, EStop:[No improvement in the 
Epoch: 99, tloss: 28.796886444091797, vloss: 32.849190, EStop:[Improvement found, cou
Epoch: 100, tloss: 42.40306091308594, vloss: 32.620056, EStop:[Improvement found, cou
Epoch: 101, tloss: 31.495004653930664, vloss: 38.031700, EStop:[No improvement in the
Epoch: 102, tloss: 40.830833435058594, vloss: 32.674007, EStop:[No improvement in the
Epoch: 103, tloss: 35.570743560791016, vloss: 33.983093, EStop:[No improvement in the
Epoch: 104, tloss: 26.948467254638672, vloss: 33.198059, EStop:[No improvement in the
Epoch: 105, tloss: 47.0742301940918, vloss: 33.876156, EStop:[No improvement in the l
Epoch: 106, tloss: 48.903175354003906, vloss: 31.343451, EStop:[Improvement found, co
Epoch: 107, tloss: 33.47947692871094, vloss: 31.096014

Epoch: 191, tloss: 18.589033126831055, vloss: 17.009523, EStop:[No improvement in the
Epoch: 192, tloss: 7.045415878295898, vloss: 15.737547, EStop:[Improvement found, cou
Epoch: 193, tloss: 27.907670974731445, vloss: 16.748480, EStop:[No improvement in the
Epoch: 194, tloss: 26.219186782836914, vloss: 16.567518, EStop:[No improvement in the
Epoch: 195, tloss: 18.017290115356445, vloss: 17.438906, EStop:[No improvement in the
Epoch: 196, tloss: 31.63317108154297, vloss: 15.744299, EStop:[No improvement in the 
Epoch: 197, tloss: 22.436180114746094, vloss: 15.661404, EStop:[Improvement found, co
Epoch: 198, tloss: 14.557696342468262, vloss: 15.826234, EStop:[No improvement in the
Epoch: 199, tloss: 16.95737648010254, vloss: 14.848233, EStop:[Improvement found, cou
Epoch: 200, tloss: 17.000244140625, vloss: 16.685390, EStop:[No improvement in the la
Epoch: 201, tloss: 14.688819885253906, vloss: 15.535754, EStop:[No improvement in the
Epoch: 202, tloss: 12.136539459228516, vloss: 14.58739

Epoch: 286, tloss: 12.237923622131348, vloss: 14.851447, EStop:[No improvement in the
Epoch: 287, tloss: 13.56518840789795, vloss: 14.050457, EStop:[No improvement in the 
Epoch: 288, tloss: 11.23833179473877, vloss: 9.931367, EStop:[No improvement in the l
Epoch: 289, tloss: 14.28852653503418, vloss: 11.461584, EStop:[No improvement in the 
Epoch: 290, tloss: 18.87738037109375, vloss: 11.912316, EStop:[No improvement in the 
Epoch: 291, tloss: 14.238763809204102, vloss: 12.226516, EStop:[No improvement in the
Epoch: 292, tloss: 8.786518096923828, vloss: 9.075766, EStop:[Improvement found, coun
Epoch: 293, tloss: 19.72392463684082, vloss: 11.697172, EStop:[No improvement in the 
Epoch: 294, tloss: 10.535595893859863, vloss: 11.720509, EStop:[No improvement in the
Epoch: 295, tloss: 12.277507781982422, vloss: 9.317815, EStop:[No improvement in the 
Epoch: 296, tloss: 14.073575019836426, vloss: 8.362963, EStop:[Improvement found, cou
Epoch: 297, tloss: 14.306373596191406, vloss: 8.373800

Epoch: 381, tloss: 11.047167778015137, vloss: 11.554057, EStop:[No improvement in the
Epoch: 382, tloss: 11.578110694885254, vloss: 6.996097, EStop:[Improvement found, cou
Epoch: 383, tloss: 17.632139205932617, vloss: 9.120420, EStop:[No improvement in the 
Epoch: 384, tloss: 10.482122421264648, vloss: 14.953797, EStop:[No improvement in the
Epoch: 385, tloss: 10.865934371948242, vloss: 8.627315, EStop:[No improvement in the 
Epoch: 386, tloss: 11.055801391601562, vloss: 9.832557, EStop:[No improvement in the 
Epoch: 387, tloss: 9.458230018615723, vloss: 7.245687, EStop:[No improvement in the l
Epoch: 388, tloss: 13.767062187194824, vloss: 8.320554, EStop:[No improvement in the 
Epoch: 389, tloss: 10.468316078186035, vloss: 7.121988, EStop:[No improvement in the 
Epoch: 390, tloss: 9.690492630004883, vloss: 7.136424, EStop:[No improvement in the l
Epoch: 391, tloss: 13.055841445922852, vloss: 6.944916, EStop:[Improvement found, cou
Epoch: 392, tloss: 9.147706985473633, vloss: 7.667737,

In [7]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.
pred = model(x_test)
score = torch.sqrt(torch.nn.functional.mse_loss(pred.flatten(), y_test))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 2.5878853797912598
